## Проект № 4

### Клиенты и счета

## Импортируем пакеты

In [1]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.types.{DataTypes, StructField}
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.functions.regexp_replace

Intitializing Scala interpreter ...

Spark Web UI available at http://146.120.224.166:4040
SparkContext available as 'sc' (version = 3.3.1, master = local[*], app id = local-1677965036128)
SparkSession available as 'spark'


import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.types.{DataTypes, StructField}
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.functions.regexp_replace


## Spark сессия

In [2]:
val spark: SparkSession = SparkSession
    .builder()
    .appName("Final project #4")
    .master("local[8]")
    .getOrCreate()

23/03/04 21:24:02 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@65fe8b7e


Импортирование дополнительных пакетов

In [3]:
import spark.implicits._

import spark.implicits._


### Даты, на которые будут строится витрины

In [4]:
val dateMartDate1 = "2020-11-01"
val dateMartDate2 = "2020-11-02"
val dateMartDate3 = "2020-11-03"
val dateMartDate4 = "2020-11-04"

dateMartDate1: String = 2020-11-01
dateMartDate2: String = 2020-11-02
dateMartDate3: String = 2020-11-03
dateMartDate4: String = 2020-11-04


In [5]:
// Все витрины будут построены на общую дату dateMartDate1
val dateMartGeneral = dateMartDate4

dateMartGeneral: String = 2020-11-04


### 1. Подготовка данных

#### a) Определение схемы данных

In [6]:
// Схема данных для таблицы клиентов
val schemaClients = new StructType()
      .add("ClientId", IntegerType, false)
      .add("ClientName", StringType, false)
      .add("Type", StringType, true)
      .add("Form", StringType, true)
      .add("RegisterDate", DateType, false)

schemaClients: org.apache.spark.sql.types.StructType = StructType(StructField(ClientId,IntegerType,false),StructField(ClientName,StringType,false),StructField(Type,StringType,true),StructField(Form,StringType,true),StructField(RegisterDate,DateType,false))


In [7]:
// Схема данных для таблицы счетов
val schemaAccounts = new StructType()
      .add("AccountId", DataTypes.IntegerType, false)
      .add("AccountNum", DataTypes.StringType, false)
      .add("ClientId", DataTypes.IntegerType, false)
      .add("DateOpen", DataTypes.DateType, false)

schemaAccounts: org.apache.spark.sql.types.StructType = StructType(StructField(AccountId,IntegerType,false),StructField(AccountNum,StringType,false),StructField(ClientId,IntegerType,false),StructField(DateOpen,DateType,false))


In [8]:
// Схема данных для таблицы банковских операций 
val schemaOperations = new StructType()
      .add("AccountDB", DataTypes.IntegerType, false)
      .add("AccountCR", DataTypes.IntegerType, false)
      .add("DateOp", DataTypes.DateType, false)
      .add("Amount", DataTypes.StringType, true)
      .add("Currency", DataTypes.StringType, false)
      .add("Comment", DataTypes.StringType, true)

schemaOperations: org.apache.spark.sql.types.StructType = StructType(StructField(AccountDB,IntegerType,false),StructField(AccountCR,IntegerType,false),StructField(DateOp,DateType,false),StructField(Amount,StringType,true),StructField(Currency,StringType,false),StructField(Comment,StringType,true))


In [9]:
// Схема данных для таблицы курса валют
val schemaRates = new StructType()
      .add("Currency", StringType, false)
      .add("Rate", StringType, false)
      .add("RateDate", DateType, false)

schemaRates: org.apache.spark.sql.types.StructType = StructType(StructField(Currency,StringType,false),StructField(Rate,StringType,false),StructField(RateDate,DateType,false))


#### b) чтение  данных и загрузка данных в dataframe

In [10]:
// Считаем файл Clients.csv и запишем прочитанное в dataframe
val dfClients = spark.read.format("csv")
      .option("header", true)
      .option("multiline", true)
      .option("delimiter", ";")
      .option("dateformat","yyyy-MM-dd")
      .option("enforceSchema", true)
      .schema(schemaClients)
      .load("hdfs://localhost:9099/user/bykovsky_nikita/project_4/Clients.csv")

dfClients: org.apache.spark.sql.DataFrame = [ClientId: int, ClientName: string ... 3 more fields]


In [11]:
// Считаем файл Account.csv и запишем прочитанное в dataframe
val dfAccounts = spark.read.format("csv")
      .option("header", true)
      .option("multiline", true)
      .option("delimiter", ";")
      .option("dateformat","yyyy-MM-dd")
      .option("enforceSchema", true)
      .schema(schemaAccounts)
      .load("hdfs://localhost:9099/user/bykovsky_nikita/project_4/Account.csv")

dfAccounts: org.apache.spark.sql.DataFrame = [AccountId: int, AccountNum: string ... 2 more fields]


In [12]:
// Считаем файл Operation.csv и запишем прочитанное в dataframe
val dfOldOperations = spark.read.format("csv")
      .option("header", true)
      .option("multiline", true)
      .option("delimiter", ";")
      .option("locale", "az-AZ")
      .option("dateformat","yyyy-MM-dd")
      .schema(schemaOperations)
      .load("hdfs://localhost:9099/user/bykovsky_nikita/project_4/Operation.csv")

dfOldOperations: org.apache.spark.sql.DataFrame = [AccountDB: int, AccountCR: int ... 4 more fields]


In [13]:
// Считаем файл Rate.csv и запишем прочитанное в dataframe
val dfOldRates = spark.read.format("csv")
      .option("header", "true")
      .option("multiline", true)
      .option("delimiter", ";")
      .option("enforceSchema", true)
      .schema(schemaRates)
      .load("hdfs://localhost:9099/user/bykovsky_nikita/project_4/Rate.csv")

dfOldRates: org.apache.spark.sql.DataFrame = [Currency: string, Rate: string ... 1 more field]


#### Просмотрим содержимое dataframe'ов

In [14]:
dfClients.show()

+--------+----------+----+----+------------+
|ClientId|ClientName|Type|Form|RegisterDate|
+--------+----------+----+----+------------+
|       1|  Клиент 1|   Ф|   -|  2020-11-01|
|       2|  Клиент 2|   Ю| ООО|  2020-11-02|
|       3|  Клиент 3|   Ф|   -|  2020-11-03|
|       4|  Клиент 4|   Ю|  ИП|  2020-11-04|
|       5|  Клиент 5|   Ф|   -|  2020-11-01|
|       6|  Клиент 6|   Ю|  АО|  2020-11-02|
|       7|  Клиент 7|   Ф|   -|  2020-11-03|
|       8|  Клиент 8|   Ю| ПАО|  2020-11-01|
|       9|  Клиент 9|   Ф|   -|  2020-11-02|
|      10| Клиент 10|   Ю| ООО|  2020-11-03|
|      11| Клиент 11|   Ф|   -|  2020-11-04|
|      12| Клиент 12|   Ю|  ИП|  2020-11-01|
|      13| Клиент 13|   Ф|   -|  2020-11-02|
|      14| Клиент 14|   Ю|  АО|  2020-11-03|
|      15| Клиент 15|   Ф|   -|  2020-11-01|
|      16| Клиент 16|   Ю| ПАО|  2020-11-02|
|      17| Клиент 17|   Ф|   -|  2020-11-03|
|      18| Клиент 18|   Ю| ООО|  2020-11-04|
|      19| Клиент 19|   Ф|   -|  2020-11-01|
|      20|

In [15]:
dfAccounts.show()

+---------+--------------------+--------+----------+
|AccountId|          AccountNum|ClientId|  DateOpen|
+---------+--------------------+--------+----------+
|        1|40702810927050000337|       1|2020-11-01|
|        2|40802810300000009067|       2|2020-11-02|
|        3|40802810300000009708|       3|2020-11-03|
|        4|40802810800000030701|       4|2020-11-04|
|        5|40802810300000011071|       5|2020-11-01|
|        6|40802810100000063339|       6|2020-11-02|
|        7|40702810823620000031|       7|2020-11-03|
|        8|40802810409260005894|       8|2020-11-04|
|        9|40802810905030000004|       9|2020-11-01|
|       10|40802810400020001097|      10|2020-11-02|
|       11|40702810717060008427|      11|2020-11-03|
|       12|40702810836260000957|      12|2020-11-04|
|       13|40702810000000150277|      13|2020-11-01|
|       14|40702810200000145833|      14|2020-11-02|
|       15|40702810200000145891|      15|2020-11-03|
|       16|40702810400000099734|      16|2020-

In [16]:
// Изменен разделитель целой и дробной части с "," на "."
val dfOperations = dfOldOperations.withColumn("Amount", regexp_replace($"Amount", ",", ".").cast(DoubleType))

dfOperations: org.apache.spark.sql.DataFrame = [AccountDB: int, AccountCR: int ... 4 more fields]


In [17]:
dfOperations.show()

+---------+---------+----------+------+--------+--------------------+
|AccountDB|AccountCR|    DateOp|Amount|Currency|             Comment|
+---------+---------+----------+------+--------+--------------------+
|        1|     4999|2020-11-01|   0.1|     RUB|а/м, а\м, автомоб...|
|        2|     5000|2020-11-02|   0.2|     USD| сою, соя, зерно,...|
|        3|     5001|2020-11-03|   0.3|      EU|сигар, табачн, та...|
|        4|     5002|2020-11-04|   0.4|     RUB|гсм, бензин, керо...|
|        5|     5003|2020-11-01|   0.5|     USD|компью, монитор, ...|
|        6|     5004|2020-11-02|   0.6|      EU|ювелир,  юв., лом...|
|        7|     5005|2020-11-03|   0.7|     RUB|хоз, бытхим, подг...|
|        8|     5006|2020-11-04|   0.8|     USD|турусл, турист, з...|
|        9|     5007|2020-11-01|   0.9|      EU|торгвыруч, оплату...|
|       10|     5008|2020-11-02|   1.0|     RUB|а/м, а\м, автомоб...|
|       11|     5009|2020-11-03|   1.1|     USD| сою, соя, зерно,...|
|       12|     5010

In [18]:
// Изменен разделитель целой и дробной части с "," на "."
val dfRates = dfOldRates.withColumn("Rate", regexp_replace($"Rate", ",", ".").cast(DoubleType))

dfRates: org.apache.spark.sql.DataFrame = [Currency: string, Rate: double ... 1 more field]


In [19]:
dfRates.show()

+--------+-----+----------+
|Currency| Rate|  RateDate|
+--------+-----+----------+
|     USD|80.23|2020-01-01|
|      EU|91.27|2020-01-01|
|     RUB|  1.0|2020-01-01|
|     USD|80.23|2020-01-02|
|      EU|91.27|2020-01-02|
|     RUB|  1.0|2020-01-02|
+--------+-----+----------+



#### Считываем файл со Списками 1 и 2

In [20]:
// Определим схему данных для локальной базы данных
val schemaLists = new StructType()
      .add("id", IntegerType, false)
      .add("list", StringType, false)

schemaLists: org.apache.spark.sql.types.StructType = StructType(StructField(id,IntegerType,false),StructField(list,StringType,false))


In [21]:
// Файл со списками находится локально
val dfLists = spark.read.format("csv")
        .option("header", "true")
        .option("multiline", true)
        .option("sep", ";")
        .option("enforceSchema", true)
        .schema(schemaLists)
        .load("Lists.csv")

dfLists: org.apache.spark.sql.DataFrame = [id: int, list: string]


In [22]:
dfLists.show()

+---+--------------------+
| id|                list|
+---+--------------------+
|  1|%а/м%, %а\м%, %ав...|
|  2|%сою%, %соя%, %зе...|
+---+--------------------+



#### d) Делаем join таблицы клиентов, счетов и operations

In [23]:
val clientsAccountsOperations = dfOperations
  .join(
    dfClients.join(dfAccounts, "ClientId")
      .select(
        $"ClientId".as("ClientDB"),
        $"AccountId".as("AccDB"),
        $"AccountNum".as("NumDB"),
        $"Type".as("TypeDB"),
        $"Form".as("FormDB")
      ),
    $"AccDB" === $"AccountDB" || $"AccDB" === $"AccountCR",
    joinType = "left"
  )
  .join(
    dfClients.join(dfAccounts, "ClientId")
      .select(
        $"ClientId".as("ClientCR"),
        $"AccountId".as("AccCR"),
        $"AccountNum".as("NumCR"),
        $"Type".as("TypeCR"),
        $"Form".as("FormCR")
      ),
    $"AccCR" === $"AccountCR",
    joinType = "left"
  )

clientsAccountsOperations: org.apache.spark.sql.DataFrame = [AccountDB: int, AccountCR: int ... 14 more fields]


In [24]:
clientsAccountsOperations.show()

+---------+---------+----------+------+--------+--------------------+--------+-----+--------------------+------+------+--------+-----+--------------------+------+------+
|AccountDB|AccountCR|    DateOp|Amount|Currency|             Comment|ClientDB|AccDB|               NumDB|TypeDB|FormDB|ClientCR|AccCR|               NumCR|TypeCR|FormCR|
+---------+---------+----------+------+--------+--------------------+--------+-----+--------------------+------+------+--------+-----+--------------------+------+------+
|        8|     5006|2020-11-04|   0.8|     USD|турусл, турист, з...|       8|    8|40802810409260005894|     Ю|   ПАО|    5006| 5006|40702810020270000072|     Ю|    АО|
|        8|     5006|2020-11-04|   0.8|     USD|турусл, турист, з...|    5006| 5006|40702810020270000072|     Ю|    АО|    5006| 5006|40702810020270000072|     Ю|    АО|
|        5|     5003|2020-11-01|   0.5|     USD|компью, монитор, ...|       5|    5|40802810300000011071|     Ф|     -|    5003| 5003|4080281052027000

#### f) Делаем join таблицы operation и курсов валют

In [25]:
// Определим курс на последнюю дату
val lastRate = dfRates.filter($"RateDate" === dfRates.select(max($"RateDate")).first().get(0))
  .select($"Currency", $"Rate")

lastRate: org.apache.spark.sql.DataFrame = [Currency: string, Rate: double]


In [26]:
// Сформируем результирующую таблицу с приведением рублевых сумм
val clientsAccountsOperationsRate = clientsAccountsOperations
  .join(lastRate, clientsAccountsOperations("Currency") === lastRate("Currency"), "left")
  .withColumn("AmountRUB", round(col("Amount") * col("Rate"), 2))
  .where($"DateOp" === dateMartGeneral)

clientsAccountsOperationsRate: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [AccountDB: int, AccountCR: int ... 17 more fields]


In [27]:
clientsAccountsOperationsRate.show()

+---------+---------+----------+------+--------+--------------------+--------+-----+--------------------+------+------+--------+-----+--------------------+------+------+--------+-----+---------+
|AccountDB|AccountCR|    DateOp|Amount|Currency|             Comment|ClientDB|AccDB|               NumDB|TypeDB|FormDB|ClientCR|AccCR|               NumCR|TypeCR|FormCR|Currency| Rate|AmountRUB|
+---------+---------+----------+------+--------+--------------------+--------+-----+--------------------+------+------+--------+-----+--------------------+------+------+--------+-----+---------+
|       32|     5030|2020-11-04|   3.2|     USD|компью, монитор, ...|      32|   32|40702810400060008556|     Ю|   ПАО|    5030| 5030|40702810512610000064|     Ю|    АО|     USD|80.23|   256.74|
|       32|     5030|2020-11-04|   3.2|     USD|компью, монитор, ...|    5030| 5030|40702810512610000064|     Ю|    АО|    5030| 5030|40702810512610000064|     Ю|    АО|     USD|80.23|   256.74|
|       40|     5038|2020

 ## 1. Построение витрины данных #1

In [28]:
val corporatePayments = clientsAccountsOperationsRate
  .groupBy($"AccDB".as("AccountID"), $"ClientDB".as("ClientId"), $"DateOp".as("CutoffDt"))
  .agg(
    round(sum(when($"AccDB" === $"AccountDB", $"AmountRUB").otherwise(0)), 2).as("PaymentAmt"),
    round(sum(when($"AccDB" === $"AccountCR", $"AmountRUB").otherwise(0)), 2).as("EnrollmentAmt"),
    round(sum(when($"AccDB" === $"AccountDB" && $"NumCR".startsWith("40702"), $"AmountRUB").otherwise(0)), 2).as("TaxAmt"),
    round(sum(when($"AccDB" === $"AccountCR" && $"NumDB".startsWith("40802"), $"AmountRUB").otherwise(0)), 2).as("ClearAmt")
  )

corporatePayments: org.apache.spark.sql.DataFrame = [AccountID: int, ClientId: int ... 5 more fields]


In [29]:
corporatePayments.show()

+---------+--------+----------+----------+-------------+---------+---------+
|AccountID|ClientId|  CutoffDt|PaymentAmt|EnrollmentAmt|   TaxAmt| ClearAmt|
+---------+--------+----------+----------+-------------+---------+---------+
|     5238|    5238|2020-11-04|       0.0|    1362176.0|      0.0|1362176.0|
|     5554|    5554|2020-11-04|       0.0|   1254341.21|      0.0|      0.0|
|     5746|    5746|2020-11-04|       0.0|    1259424.8|      0.0|      0.0|
|      960|     960|2020-11-04| 1386944.0|          0.0|1386944.0|      0.0|
|     5994|    5994|2020-11-04|       0.0|    1388182.4|      0.0|      0.0|
|     1104|    1104|2020-11-04| 1391897.6|          0.0|1391897.6|      0.0|
|     6170|    6170|2020-11-04|       0.0|    886197.16|      0.0|      0.0|
|     1452|    1452|2020-11-04| 1403868.8|          0.0|      0.0|      0.0|
|     1544|    1544|2020-11-04| 895635.91|          0.0|895635.91|      0.0|
|     1796|    1796|2020-11-04| 902029.91|          0.0|      0.0|      0.0|

#### Запись витрины corporatePayments в файл формата parquet

In [30]:
// Запишем витрину corporate_payments в parquet-файл
corporatePayments.write
    .format("parquet")
    .option("path", "data_marts/" + dateMartGeneral + "/corporate_payments")
    .mode("Overwrite")
    .save()

 ## 2. Построение витрины данных #2

In [31]:
val corporateAccount = corporatePayments
    .join(dfClients.join(dfAccounts, Seq("clientId"))
    .select($"AccountNum", $"AccountID", $"DateOpen", $"ClientName"), Seq("AccountID"))
    .withColumn("TotalAmt", round($"PaymentAmt" + $"EnrollmentAmt", 2))
    .select($"AccountID", $"AccountNum", $"DateOpen", $"ClientId", $"ClientName", $"TotalAmt", $"CutoffDt")
    .orderBy($"AccountID", $"CutoffDt")

corporateAccount: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [AccountID: int, AccountNum: string ... 5 more fields]


In [32]:
corporateAccount.show()

+---------+--------------------+----------+--------+----------+----------+----------+
|AccountID|          AccountNum|  DateOpen|ClientId|ClientName|  TotalAmt|  CutoffDt|
+---------+--------------------+----------+--------+----------+----------+----------+
|        3|40802810300000009708|2020-11-03|       3|  Клиент 3|1110291.49|2020-11-04|
|        4|40802810800000030701|2020-11-04|       4|  Клиент 4|1239725.91|2020-11-04|
|        7|40702810823620000031|2020-11-03|       7|  Клиент 7| 1698195.2|2020-11-04|
|        8|40802810409260005894|2020-11-04|       8|  Клиент 8| 856662.98|2020-11-04|
|       11|40702810717060008427|2020-11-03|      11| Клиент 11|1504707.72|2020-11-04|
|       12|40702810836260000957|2020-11-04|      12| Клиент 12| 1354332.8|2020-11-04|
|       15|40702810200000145891|2020-11-03|      15| Клиент 15|1110595.97|2020-11-04|
|       16|40702810400000099734|2020-11-04|      16| Клиент 16|1240043.63|2020-11-04|
|       19|40702810400000100092|2020-11-03|      19| К

#### Запись витрины corporate_account в файл формата parquet

In [33]:
// Запишем витрину в parquet-файл
corporateAccount.write
    .format("parquet")
    .option("path", "data_marts/" + dateMartGeneral + "/corporate_account")
    .mode("Overwrite")
    .save()

 ## 4. Построение витрины данных #3

In [34]:
val corporateInfo = corporateAccount
    .join(dfClients.join(dfAccounts, "ClientId")
        .select($"ClientId".alias("Clid"),
                $"ClientName".alias("ClName"),
                $"Type",
                $"Form",
                $"RegisterDate"),
            $"Clid" === $"ClientId")

corporateInfo: org.apache.spark.sql.DataFrame = [AccountID: int, AccountNum: string ... 10 more fields]


In [35]:
corporateInfo.show()

+---------+--------------------+----------+--------+-----------+----------+----------+----+-----------+----+----+------------+
|AccountID|          AccountNum|  DateOpen|ClientId| ClientName|  TotalAmt|  CutoffDt|Clid|     ClName|Type|Form|RegisterDate|
+---------+--------------------+----------+--------+-----------+----------+----------+----+-----------+----+----+------------+
|     7628|40702810240420009572|2020-11-04|    7628|Клиент 7628|1050005.24|2020-11-04|7628|Клиент 7628|   Ю|  ИП|  2020-11-01|
|     7628|40702810240420009572|2020-11-04|    7628|Клиент 7628|1050005.24|2020-11-04|7628|Клиент 7628|   Ю|  ИП|  2020-11-01|
|    15996|40802810403030000247|2020-11-04|    5996|Клиент 5996| 1904182.4|2020-11-04|5996|Клиент 5996|   Ю|  ИП|  2020-11-04|
|    15996|40802810403030000247|2020-11-04|    5996|Клиент 5996| 1904182.4|2020-11-04|5996|Клиент 5996|   Ю|  ИП|  2020-11-04|
|     3843|40702810416450003609|2020-11-03|    3843|Клиент 3843|1207723.81|2020-11-04|3843|Клиент 3843|   Ф|   

#### Запись витрины corporateInfo в файл формата parquet

In [36]:
corporateInfo.write
    .format("parquet")
    .option("path", "data_marts/" + dateMartGeneral + "/corporate_info")
    .mode("Overwrite")
    .save()

In [37]:
spark.stop